In [2]:
import pandas as pd
import numpy as np

In [3]:
countries={'Albania': 'AL', 'Armenia': 'AM', 'Austria': 'AT', 'Azerbaijan': 'AZ', 'Belarus': 'BY','Belgium': 'BE', 'Bosnia and Herz.': 'BA', 'Bulgaria': 'BG', 'Croatia': 'HR', 'Cyprus': 'CY', 'Switzerland': 'CH', 'Czech Republic': 'CZ', 'Germany': 'DE', 'Georgia': 'GE', 'Denmark': 'DK', 'Estonia': 'EE', 'Spain': 'ES', 'Finland': 'FI', 'France': 'FR', 'United Kingdom': 'UK', 'Greece': 'GR', 'Hungary': 'HU', 'Iceland': 'IS', 'Ireland': 'IE', 'Italy': 'IT', 'Lithuania': 'LT', 'Luxembourg': 'LU', 'Latvia': 'LV', 'Montenegro': 'ME', 'North Macedonia': 'MK', 'Netherlands': 'NL', 'Norway': 'NO', 'Poland': 'PL', 'Portugal': 'PT', 'Romania': 'RO', 'Serbia': 'RS', 'Sweden': 'SE', 'Slovenia': 'SI', 'Slovakia': 'SK', 'Kosovo': 'XK', 'Turkey': 'TR', 'Ukraine': 'UA', 'Russia': 'RU',  'Malta': 'MT', 'Moldova': 'MD'}

In [8]:
fuels=['Biomass', 'Fossil Brown coal/Lignite', 'Fossil Coal-derived gas', 'Fossil Gas', 'Fossil Hard coal', 'Fossil Oil', 'Fossil Oil shale', 'Fossil Peat', 'Geothermal', 'Hydro Pumped Storage', 'Hydro Pumped Storage', 'Hydro Run-of-river and poundage', 'Hydro Water Reservoir', 'Marine', 'Nuclear', 'Other', 'Other renewable', 'Solar', 'Waste', 'Wind Offshore', 'Wind Onshore']

# Preparing Load Data

## Making the first,second,third and fourth columns of the dataframe as date,month,year and time

In [9]:
temp=pd.read_csv('../Data Sources/ENTSO-E/Load/Germany.csv')
load_data=pd.DataFrame()
load_data['Day']=temp['Time (CET)'].str[:2]
load_data['Month']=temp['Time (CET)'].str[3:5]
load_data['Year']=temp['Time (CET)'].str[6:10]
load_data['Time']=temp['Time (CET)'].str[11:16]+' - '+temp['Time (CET)'].str[29:35]

## Getting the load data of all countries into a single dataframe

In [10]:
for country,abbr in countries.items():
    temp=pd.read_csv(f'../Data Sources/ENTSO-E/Load/{country}.csv')
    load_data[f'{abbr}']=temp[f'Actual Total Load [MW] - {country} ({abbr})']

In [11]:
load_data.head()

,Day,Month,Year,Time,AL,AM,AT,AZ,BY,BE,...,RS,SE,SI,SK,XK,TR,UA,RU,MT,MD
0,01,01,2019,00:00 - 00:15,NaN,n/e,6139.0,n/e,n/e,9122.0,...,5290.0,14597.0,1164.0,2835.0,n/e,n/e,16997.0,n/e,n/e,n/e
1,01,01,2019,00:15 - 00:30,NaN,n/e,6108.0,n/e,n/e,9060.0,...,5282.0,14375.0,1129.0,2744.0,n/e,n/e,16726.0,n/e,n/e,n/e
2,01,01,2019,00:30 - 00:45,NaN,n/e,6054.0,n/e,n/e,8964.0,...,5067.0,14153.0,1084.0,2679.0,n/e,n/e,16572.0,n/e,n/e,n/e
3,01,01,2019,00:45 - 01:00,NaN,n/e,5999.0,n/e,n/e,8747.0,...,4739.0,13896.0,1038.0,2608.0,n/e,n/e,16449.0,n/e,n/e,n/e
4,01,01,2019,01:00 - 01:15,NaN,n/e,5976.0,n/e,n/e,8729.0,...,4445.0,13955.0,1018.0,2602.0,n/e,n/e,16300.0,n/e,n/e,n/e


# Preparing Generation Data

In [12]:
generation_data=pd.DataFrame()
generation_data[['Day','Month','Year','Time']]=load_data[['Day','Month','Year','Time']]

In [14]:
for country,abbr in countries.items():
    temp=pd.read_csv(f'../Data Sources/ENTSO-E/Generation/{country}.csv')
    for fuel in fuels:
        generation_data[f'{fuel} - {abbr}']=temp[f'{fuel}  - Actual Aggregated [MW]']

C:\Users\lovin\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3169: DtypeWarning: Columns (19) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [16]:
generation_data.head()

,Day,Month,Year,Time,Biomass - AL,Fossil Brown coal/Lignite - AL,Fossil Coal-derived gas - AL,Fossil Gas - AL,Fossil Hard coal - AL,Fossil Oil - AL,...,Hydro Run-of-river and poundage - MD,Hydro Water Reservoir - MD,Marine - MD,Nuclear - MD,Other - MD,Other renewable - MD,Solar - MD,Waste - MD,Wind Offshore - MD,Wind Onshore - MD
0,01,01,2019,00:00 - 00:15,n/e,n/e,n/e,n/e,n/e,n/e,...,n/e,n/e,n/e,n/e,n/e,n/e,n/e,n/e,n/e,n/e
1,01,01,2019,00:15 - 00:30,n/e,n/e,n/e,n/e,n/e,n/e,...,n/e,n/e,n/e,n/e,n/e,n/e,n/e,n/e,n/e,n/e
2,01,01,2019,00:30 - 00:45,n/e,n/e,n/e,n/e,n/e,n/e,...,n/e,n/e,n/e,n/e,n/e,n/e,n/e,n/e,n/e,n/e
3,01,01,2019,00:45 - 01:00,n/e,n/e,n/e,n/e,n/e,n/e,...,n/e,n/e,n/e,n/e,n/e,n/e,n/e,n/e,n/e,n/e
4,01,01,2019,01:00 - 01:15,n/e,n/e,n/e,n/e,n/e,n/e,...,n/e,n/e,n/e,n/e,n/e,n/e,n/e,n/e,n/e,n/e
